## Imports

In [ ]:
import os
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
from scipy.stats import ks_2samp
from plotchecker import LinePlotChecker, ScatterPlotChecker, BarPlotChecker
import seaborn as sns
import requests

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler,OrdinalEncoder
from sklearn.metrics import precision_score, recall_score, precision_recall_curve
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier, LGBMRegressor 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.metrics import mean_absolute_percentage_error


import lightgbm as lgb

from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

C:\Users\catar\AppData\Local\Temp\ipykernel_16648\1405592005.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Functions

In [3]:
def grouped_weighted_mape_bias(df, group_cols, y_true_col, y_pred_col, quantity_col):
    # Function to compute weighted MAPE & BIAS
    grouped = df.groupby(group_cols).apply(lambda g: pd.Series({
        "mape": np.sum(np.abs(g[y_true_col] - g[y_pred_col]) * g[quantity_col]) / np.sum(g[y_true_col] * g[quantity_col]),
        "bias": np.sum((g[y_pred_col] - g[y_true_col]) * g[quantity_col]) / np.sum(g[quantity_col])
    }), include_groups=False)
    
    return grouped.reset_index()

In [4]:
def plot_feature_importance(model, feature_names, title):
    """Plots feature importance from LightGBM model after preprocessing"""

    # Extract LightGBM feature importance
    importance = model.named_steps["model"].feature_importances_

    # Ensure feature names align with importance array
    if len(feature_names) != len(importance):
        print(f"Warning: Feature name length ({len(feature_names)}) does not match importance length ({len(importance)}). Adjusting...")
        feature_names = feature_names[:len(importance)]  # Trim feature names if mismatch occurs

    # Create DataFrame
    importance_df = pd.DataFrame({
        "feature": feature_names,
        "importance": importance
    })

    # Sort by importance
    importance_df = importance_df.sort_values(by="importance", ascending=False)

    # Plot
    plt.figure(figsize=(10, 10))
    sns.barplot(data=importance_df, x="importance", y="feature", hue="feature", legend=False, palette="coolwarm")
    plt.title(title)
    plt.xlabel("Importance")
    plt.ylabel("Feature")
    plt.show()



## Read data

In [5]:
dtype_dict = {
    'structure_level_4': 'object',
    'structure_level_3': 'object',
    'structure_level_2': 'object',
    'structure_level_1': 'object',
    'sku': 'int64',
    'competitor': 'object',
    'chain_campaign': 'object',
    'quantity': 'float64',
    'pvp_was': 'float64',
    'discount': 'float64',
    'flag_promo': 'float64',
    'leaflet': 'object',
    'pvp_is': 'float64',
    'flg_filled_gap': 'float64',
    'month': 'int32',
    'day_of_week': 'int32',
    'week_of_month': 'int64',
    'holiday': 'object',
    'holiday_importance': 'int64',
    'apparent_temperature_mean': 'int64',
    'precipitation_sum': 'int64',
    'abc': 'object',
    'seil': 'object',
    'xyz': 'object',
    'pvp_is_lag_1': 'float64',
    'pvp_is_lag_7': 'float64',
    'pvp_is_lag_30': 'float64',
    'discount_lag_1': 'float64',
    'discount_lag_7': 'float64',
    'discount_lag_30': 'float64',
    'days_since_last_promo': 'float64',
    'rolling_mean_7': 'float64',
    'rolling_mean_30': 'float64',
    'avg_discount_w_L3': 'float64',
    'promo_part_w_L3': 'float64',
    'avg_discount_w_sku': 'float64',
    'promo_part_w_sku': 'float64',
    'discount_chain': 'float64',
    'leaflet_chain': 'object'
}

In [ ]:
df_input = pd.read_csv("prepared_data/data.csv",dtype=dtype_dict, parse_dates=['date'])
df_input['avg_discount_w_L3'] = df_input['avg_discount_w_L3'].round(2)
df_input['promo_part_w_L3'] = df_input['promo_part_w_L3'].round(2)
df_input['avg_discount_w_sku'] = df_input['avg_discount_w_sku'].round(2)
df_input['promo_part_w_sku'] = df_input['promo_part_w_sku'].round(2)

## Modelling

### ARIMA / SARIMA

In [7]:
list_structure_level_1=df_input['structure_level_1'].sort_values().unique()
list_competitor=df_input['competitor'].sort_values().unique()

In [50]:
df = df_input[['sku', 'date','competitor', ## keys
                'pvp_is', ## target 
                'quantity', ## used for evaluation
                ## features:
                'structure_level_3', 'structure_level_1', #'structure_level_3', 'structure_level_2', 'structure_level_1', 
                'chain_campaign', 'holiday_importance','avg_discount_w_L3','promo_part_w_L3',
                'month', 'day_of_week', 'week_of_month',
                'apparent_temperature_mean', 'precipitation_sum', 
                'abc', 'seil', 'xyz',
                'discount_chain', 'leaflet_chain']]

In [ ]:
# Ensure the models directory exists
TMP_DIR = "models_sarima"
os.makedirs(TMP_DIR, exist_ok=True)

# Ensure date is in datetime format
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")
train_cutoff = pd.to_datetime("2024-06-04")

results = []
all_predictions = []

# Loop through structure_level_1 & competitors
for struct1 in list_structure_level_1:
    for comp in list_competitor:
        print("***************************************************************************************************")
        print(f"Structure_level_1: {struct1}, Competitor: {comp}")

        ## 1️. Filter dataset
        df_filtered = df[(df['structure_level_1'] == struct1) & (df['competitor'] == comp)].copy()

        for sku in df_filtered['sku'].unique():
            df_sku = df_filtered[df_filtered['sku'] == sku].sort_values('date')

            if len(df_sku) < 30:
                print("sku ", sku, " and ", comp, " too short <30: ", len(df_sku))
                continue  # skip too short series

            df_sku = df_sku[['date', 'pvp_is', 'quantity']].dropna().drop_duplicates()
            df_sku['date'] = pd.to_datetime(df_sku['date'])
            df_sku = df_sku.set_index('date')
            df_sku = df_sku.asfreq('D')
            df_sku = df_sku.ffill() ## there should be no nan at this step -> just doing to prevent

            # Train-test split
            df_train = df_sku[df_sku.index < train_cutoff].dropna()
            df_test = df_sku[df_sku.index >= train_cutoff].dropna()

            if len(df_test) == 0 or len(df_train) < 14:
                print("sku ", sku, " and ", comp, " too short len(df_test)==0: ", len(df_test), " or len(df_train)<14: ",len(df_train))
                continue
            try:
                # Fit auto-SARIMA model
                model = auto_arima(
                    df_train['pvp_is'],
                    start_p=1, start_q=1, max_p=3, max_q=3,
                    d=None,           # Let it infer if differencing is needed
                    seasonal=True,
                    m=7,              # Weekly pattern
                    start_P=0, start_Q=0, max_P=2, max_Q=2,
                    D=1,
                    suppress_warnings=True,
                    stepwise=True
                )

                forecast = model.predict(n_periods=len(df_test))
                if np.all(np.isnan(forecast)):
                    print(f"Forecast returned all NaNs for SKU {sku}")
                    continue

                # Evaluate
                df_test = df_test.copy()
                df_test['pvp_is_pred'] = forecast
                df_test['sku'] = sku
                df_test['competitor'] = comp
                df_test['structure_level_1'] = struct1
                df_test = df_test.reset_index().rename(columns={'index': 'date'})

                all_predictions.append(df_test)

                df_test['abs_error'] = np.abs(df_test['pvp_is_pred'] - df_test['pvp_is'])
                df_test['error'] = df_test['pvp_is_pred'] - df_test['pvp_is']

                # Use quantity for weighting
                df_test['weight'] = df_test['quantity'].fillna(0)

                total_weight = df_test['weight'].sum()
                if total_weight == 0:
                    mape = np.nan
                    bias = np.nan
                else:
                    mape = (df_test['abs_error'] / df_test['pvp_is'] * df_test['weight']).sum() / total_weight
                    bias = (df_test['error'] * df_test['weight']).sum() / total_weight

                results.append({
                    'structure_level_1': struct1,
                    'competitor': comp,
                    'sku': sku,
                    'mape': mape,
                    'bias': bias,
                    'model_params': model.get_params()
                })

                # Retrain on full data
                model.fit(df_sku['pvp_is'])
                model_path = os.path.join(TMP_DIR, f"sarima_struct{struct1}_{comp}_sku{sku}.pkl")
                joblib.dump(model, model_path)

            except Exception as e:
                print(f" Failed for SKU {sku}: {e}")
                continue
        
        if len(results)>0:
            # Combine and summarize results
            df_predictions = pd.concat(all_predictions, ignore_index=True)
            df_sarima_results = pd.DataFrame(results)
            
    print("\n=== Summary by Structure & Competitor ===")
    print(grouped_weighted_mape_bias(df_predictions, ['structure_level_1', 'competitor'], 'pvp_is', 'pvp_is_pred', 'quantity'))

    print("\n Models and predictions saved.")

C:\Users\catar\AppData\Local\Temp\ipykernel_18420\2830822718.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"] = pd.to_datetime(df["date"])


***************************************************************************************************
Structure_level_1: 1, Competitor: competitorB
sku  2821  and  competitorB  too short <30:  0
sku  2734  and  competitorB  too short <30:  0
sku  2786  and  competitorB  too short <30:  0
sku  2754  and  competitorB  too short <30:  0
sku  2664  and  competitorB  too short <30:  0
sku  2992  and  competitorB  too short <30:  0
sku  2993  and  competitorB  too short <30:  0
sku  2822  and  competitorB  too short <30:  0
sku  2699  and  competitorB  too short <30:  0


c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(2,0,1)(2,1,0)[7] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "c:\Users\catar\AppData\Local\Pro

sku  2712  and  competitorB  too short <30:  0
sku  2669  and  competitorB  too short <30:  0


c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\auto.py:444: UserWarning: Input time-series is completely constant; returning a (0, 0, 0) ARMA.
  warnings.warn('Input time-series is completely constant; '


sku  2672  and  competitorB  too short <30:  0
sku  2673  and  competitorB  too short <30:  0
sku  2779  and  competitorB  too short <30:  0
sku  2881  and  competitorB  too short <30:  0


c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\_auto_solvers.py:524: ModelFitWarning: Error fitting  ARIMA(3,0,2)(2,1,0)[7] intercept (if you do not want to see these warnings, run with error_action="ignore").
Traceback:
Traceback (most recent call last):
  File "c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\_auto_solvers.py", line 508, in _fit_candidate_model
    fit.fit(y, X=X, **fit_params)
  File "c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\arima.py", line 603, in fit
    self._fit(y, X, **fit_args)
  File "c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\arima.py", line 524, in _fit
    fit, self.arima_res_ = _fit_wrapper()
  File "c:\Users\catar\AppData\Local\Programs\Python\Python310\lib\site-packages\pmdarima\arima\arima.py", line 510, in _fit_wrapper
    fitted = arima.fit(
  File "c:\Users\catar\AppData\Local\Pro

sku  2628  and  competitorB  too short <30:  0
sku  2941  and  competitorB  too short <30:  0
sku  2939  and  competitorB  too short <30:  0
sku  2940  and  competitorB  too short <30:  0
sku  2942  and  competitorB  too short <30:  0
sku  2623  and  competitorB  too short <30:  0
sku  2620  and  competitorB  too short <30:  0
sku  2605  and  competitorB  too short <30:  0
sku  2602  and  competitorB  too short <30:  0

=== Summary by Structure & Competitor ===
  structure_level_1   competitor      mape     bias
0                 1  competitorB  0.060523 -1.20181

 Models and predictions saved.


In [53]:
# df_predictions.to_csv("20250512_df_predictions_auto_arima_57_skus_compA_structL1_1.csv")
df_predictions.to_csv("20250512_df_predictions_auto_arima_57_skus_compB_structL1_1.csv")

In [36]:
if len(results)>0:
        # Combine and summarize results
        df_predictions = pd.concat(all_predictions, ignore_index=True)
        df_sarima_results = pd.DataFrame(results)
        
print("\n=== Summary by Structure & Competitor ===")
print(grouped_weighted_mape_bias(df_predictions, ['structure_level_1', 'competitor'], 'pvp_is', 'pvp_is_pred', 'quantity'))
print("\n nº skus executed: ", df_predictions['sku'].drop_duplicates().count())

print("\n Models and predictions saved.")


=== Summary by Structure & Competitor ===
  structure_level_1   competitor      mape      bias
0                 1  competitorA  0.119936 -2.381066

 nº skus executed:  57

 Models and predictions saved.
